https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-an-mnist-handwritten-digits-from-scratch-in-keras/

In [ ]:

# example of training a gan on mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
 
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
	model = Sequential()
	model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model
 
# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 128)))
	# upsample to 14x14
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 28x28
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
	return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model
 
# load and prepare mnist training images
def load_real_samples():
	# load mnist dataset
	(trainX, _), (_, _) = load_data()
	# expand to 3d, e.g. add channels dimension
	X = expand_dims(trainX, axis=-1)
	# convert from unsigned ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [0,1]
	X = X / 255.0
	return X
 
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y
 
# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()
 
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch + 1)
	g_model.save(filename)
 
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# create training set for the discriminator
			X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
			# update discriminator model weights
			d_loss, _ = d_model.train_on_batch(X, y)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)
 
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

>1, 1/234, d=0.677, g=0.737
>1, 2/234, d=0.671, g=0.749
>1, 3/234, d=0.663, g=0.773
>1, 4/234, d=0.654, g=0.792
>1, 5/234, d=0.643, g=0.809
>1, 6/234, d=0.644, g=0.824
>1, 7/234, d=0.636, g=0.843
>1, 8/234, d=0.627, g=0.854
>1, 9/234, d=0.623, g=0.861
>1, 10/234, d=0.618, g=0.860
>1, 11/234, d=0.624, g=0.849
>1, 12/234, d=0.624, g=0.830
>1, 13/234, d=0.631, g=0.800
>1, 14/234, d=0.636, g=0.772
>1, 15/234, d=0.640, g=0.752
>1, 16/234, d=0.640, g=0.736
>1, 17/234, d=0.640, g=0.725
>1, 18/234, d=0.632, g=0.718
>1, 19/234, d=0.629, g=0.712
>1, 20/234, d=0.629, g=0.709
>1, 21/234, d=0.614, g=0.706
>1, 22/234, d=0.609, g=0.705
>1, 23/234, d=0.602, g=0.704
>1, 24/234, d=0.596, g=0.704
>1, 25/234, d=0.585, g=0.704
>1, 26/234, d=0.576, g=0.704
>1, 27/234, d=0.568, g=0.704
>1, 28/234, d=0.559, g=0.704
>1, 29/234, d=0.553, g=0.704
>1, 30/234, d=0.538, g=0.705
>1, 31/234, d=0.537, g=0.706
>1, 32/234, d=0.516, g=0.705
>1, 33/234, d=0.514, g=0.706
>1, 34/234, d=0.497, g=0.707
>1, 35/234, d=0.485, g=

>2, 46/234, d=0.660, g=0.774
>2, 47/234, d=0.691, g=0.732
>2, 48/234, d=0.702, g=0.712
>2, 49/234, d=0.718, g=0.703
>2, 50/234, d=0.704, g=0.697
>2, 51/234, d=0.724, g=0.678
>2, 52/234, d=0.736, g=0.682
>2, 53/234, d=0.749, g=0.663
>2, 54/234, d=0.760, g=0.657
>2, 55/234, d=0.775, g=0.670
>2, 56/234, d=0.799, g=0.661
>2, 57/234, d=0.790, g=0.694
>2, 58/234, d=0.794, g=0.667
>2, 59/234, d=0.807, g=0.654
>2, 60/234, d=0.789, g=0.634
>2, 61/234, d=0.782, g=0.647
>2, 62/234, d=0.760, g=0.653
>2, 63/234, d=0.760, g=0.667
>2, 64/234, d=0.774, g=0.679
>2, 65/234, d=0.760, g=0.683
>2, 66/234, d=0.745, g=0.720
>2, 67/234, d=0.740, g=0.704
>2, 68/234, d=0.727, g=0.730
>2, 69/234, d=0.685, g=0.734
>2, 70/234, d=0.710, g=0.742
>2, 71/234, d=0.681, g=0.741
>2, 72/234, d=0.681, g=0.743
>2, 73/234, d=0.650, g=0.744
>2, 74/234, d=0.628, g=0.752
>2, 75/234, d=0.621, g=0.765
>2, 76/234, d=0.609, g=0.780
>2, 77/234, d=0.619, g=0.799
>2, 78/234, d=0.593, g=0.816
>2, 79/234, d=0.604, g=0.830
>2, 80/234, d=

>3, 91/234, d=0.681, g=0.731
>3, 92/234, d=0.681, g=0.731
>3, 93/234, d=0.697, g=0.726
>3, 94/234, d=0.678, g=0.722
>3, 95/234, d=0.687, g=0.724
>3, 96/234, d=0.687, g=0.718
>3, 97/234, d=0.686, g=0.732
>3, 98/234, d=0.689, g=0.720
>3, 99/234, d=0.701, g=0.735
>3, 100/234, d=0.690, g=0.732
>3, 101/234, d=0.685, g=0.730
>3, 102/234, d=0.683, g=0.709
>3, 103/234, d=0.709, g=0.735
>3, 104/234, d=0.689, g=0.739
>3, 105/234, d=0.693, g=0.705
>3, 106/234, d=0.689, g=0.713
>3, 107/234, d=0.701, g=0.738
>3, 108/234, d=0.694, g=0.722
>3, 109/234, d=0.692, g=0.722
>3, 110/234, d=0.698, g=0.716
>3, 111/234, d=0.712, g=0.734
>3, 112/234, d=0.685, g=0.725
>3, 113/234, d=0.687, g=0.728
>3, 114/234, d=0.694, g=0.714
>3, 115/234, d=0.706, g=0.733
>3, 116/234, d=0.683, g=0.699
>3, 117/234, d=0.681, g=0.715
>3, 118/234, d=0.688, g=0.729
>3, 119/234, d=0.694, g=0.715
>3, 120/234, d=0.682, g=0.720
>3, 121/234, d=0.693, g=0.722
>3, 122/234, d=0.690, g=0.713
>3, 123/234, d=0.694, g=0.712
>3, 124/234, d=0.70

>4, 134/234, d=0.688, g=0.714
>4, 135/234, d=0.688, g=0.730
>4, 136/234, d=0.688, g=0.743
>4, 137/234, d=0.687, g=0.724
>4, 138/234, d=0.682, g=0.727
>4, 139/234, d=0.682, g=0.725
>4, 140/234, d=0.676, g=0.733
>4, 141/234, d=0.680, g=0.729
>4, 142/234, d=0.688, g=0.734
>4, 143/234, d=0.675, g=0.730
>4, 144/234, d=0.679, g=0.717
>4, 145/234, d=0.684, g=0.732
>4, 146/234, d=0.684, g=0.741
>4, 147/234, d=0.680, g=0.746
>4, 148/234, d=0.677, g=0.739
>4, 149/234, d=0.692, g=0.738
>4, 150/234, d=0.693, g=0.730
>4, 151/234, d=0.669, g=0.739
>4, 152/234, d=0.684, g=0.725
>4, 153/234, d=0.698, g=0.730
>4, 154/234, d=0.693, g=0.727
>4, 155/234, d=0.682, g=0.723
>4, 156/234, d=0.685, g=0.727
>4, 157/234, d=0.684, g=0.725
>4, 158/234, d=0.683, g=0.719
>4, 159/234, d=0.676, g=0.732
>4, 160/234, d=0.682, g=0.726
>4, 161/234, d=0.670, g=0.746
>4, 162/234, d=0.677, g=0.734
>4, 163/234, d=0.696, g=0.741
>4, 164/234, d=0.680, g=0.745
>4, 165/234, d=0.676, g=0.741
>4, 166/234, d=0.679, g=0.726
>4, 167/23

>5, 177/234, d=0.677, g=0.708
>5, 178/234, d=0.678, g=0.715
>5, 179/234, d=0.691, g=0.712
>5, 180/234, d=0.682, g=0.721
>5, 181/234, d=0.684, g=0.718
>5, 182/234, d=0.689, g=0.714
>5, 183/234, d=0.698, g=0.712
>5, 184/234, d=0.679, g=0.723
>5, 185/234, d=0.693, g=0.705
>5, 186/234, d=0.700, g=0.725
>5, 187/234, d=0.685, g=0.716
>5, 188/234, d=0.716, g=0.696
>5, 189/234, d=0.714, g=0.703
>5, 190/234, d=0.701, g=0.710
>5, 191/234, d=0.701, g=0.714
>5, 192/234, d=0.708, g=0.733
>5, 193/234, d=0.694, g=0.715
>5, 194/234, d=0.717, g=0.709
>5, 195/234, d=0.709, g=0.698
>5, 196/234, d=0.711, g=0.715
>5, 197/234, d=0.710, g=0.705
>5, 198/234, d=0.707, g=0.695
>5, 199/234, d=0.711, g=0.708
>5, 200/234, d=0.696, g=0.700
>5, 201/234, d=0.714, g=0.696
>5, 202/234, d=0.708, g=0.688
>5, 203/234, d=0.701, g=0.707
>5, 204/234, d=0.709, g=0.700
>5, 205/234, d=0.714, g=0.701
>5, 206/234, d=0.704, g=0.707
>5, 207/234, d=0.697, g=0.713
>5, 208/234, d=0.702, g=0.718
>5, 209/234, d=0.711, g=0.728
>5, 210/23

>6, 220/234, d=0.730, g=0.700
>6, 221/234, d=0.740, g=0.691
>6, 222/234, d=0.757, g=0.677
>6, 223/234, d=0.739, g=0.676
>6, 224/234, d=0.736, g=0.658
>6, 225/234, d=0.730, g=0.657
>6, 226/234, d=0.748, g=0.656
>6, 227/234, d=0.743, g=0.663
>6, 228/234, d=0.718, g=0.683
>6, 229/234, d=0.720, g=0.699
>6, 230/234, d=0.710, g=0.723
>6, 231/234, d=0.699, g=0.747
>6, 232/234, d=0.686, g=0.763
>6, 233/234, d=0.679, g=0.806
>6, 234/234, d=0.667, g=0.824
>7, 1/234, d=0.664, g=0.848
>7, 2/234, d=0.653, g=0.855
>7, 3/234, d=0.640, g=0.865
>7, 4/234, d=0.644, g=0.869
>7, 5/234, d=0.630, g=0.858
>7, 6/234, d=0.635, g=0.838
>7, 7/234, d=0.639, g=0.808
>7, 8/234, d=0.652, g=0.787
>7, 9/234, d=0.657, g=0.770
>7, 10/234, d=0.671, g=0.740
>7, 11/234, d=0.666, g=0.729
>7, 12/234, d=0.671, g=0.718
>7, 13/234, d=0.678, g=0.726
>7, 14/234, d=0.688, g=0.721
>7, 15/234, d=0.694, g=0.710
>7, 16/234, d=0.693, g=0.704
>7, 17/234, d=0.685, g=0.706
>7, 18/234, d=0.708, g=0.706
>7, 19/234, d=0.701, g=0.701
>7, 20/2

>8, 30/234, d=0.686, g=0.713
>8, 31/234, d=0.682, g=0.704
>8, 32/234, d=0.688, g=0.704
>8, 33/234, d=0.687, g=0.694
>8, 34/234, d=0.686, g=0.685
>8, 35/234, d=0.684, g=0.685
>8, 36/234, d=0.693, g=0.682
>8, 37/234, d=0.693, g=0.684
>8, 38/234, d=0.700, g=0.681
>8, 39/234, d=0.684, g=0.676
>8, 40/234, d=0.698, g=0.683
>8, 41/234, d=0.696, g=0.674
>8, 42/234, d=0.693, g=0.702
>8, 43/234, d=0.692, g=0.692
>8, 44/234, d=0.701, g=0.699
>8, 45/234, d=0.701, g=0.704
>8, 46/234, d=0.700, g=0.694
>8, 47/234, d=0.702, g=0.689
>8, 48/234, d=0.708, g=0.687
>8, 49/234, d=0.706, g=0.690
>8, 50/234, d=0.702, g=0.680
>8, 51/234, d=0.706, g=0.687
>8, 52/234, d=0.709, g=0.675
>8, 53/234, d=0.701, g=0.679
>8, 54/234, d=0.696, g=0.686
>8, 55/234, d=0.696, g=0.688
>8, 56/234, d=0.697, g=0.696
>8, 57/234, d=0.713, g=0.705
>8, 58/234, d=0.704, g=0.706
>8, 59/234, d=0.695, g=0.712
>8, 60/234, d=0.707, g=0.719
>8, 61/234, d=0.699, g=0.729
>8, 62/234, d=0.701, g=0.726
>8, 63/234, d=0.699, g=0.718
>8, 64/234, d=

>9, 75/234, d=0.690, g=0.719
>9, 76/234, d=0.695, g=0.719
>9, 77/234, d=0.692, g=0.712
>9, 78/234, d=0.688, g=0.696
>9, 79/234, d=0.698, g=0.693
>9, 80/234, d=0.703, g=0.682
>9, 81/234, d=0.702, g=0.680
>9, 82/234, d=0.700, g=0.668
>9, 83/234, d=0.691, g=0.668
>9, 84/234, d=0.691, g=0.666
>9, 85/234, d=0.688, g=0.654
>9, 86/234, d=0.691, g=0.664
>9, 87/234, d=0.698, g=0.655
>9, 88/234, d=0.698, g=0.668
>9, 89/234, d=0.700, g=0.681
>9, 90/234, d=0.695, g=0.708
>9, 91/234, d=0.698, g=0.703
>9, 92/234, d=0.694, g=0.712
>9, 93/234, d=0.687, g=0.718
>9, 94/234, d=0.687, g=0.721
>9, 95/234, d=0.692, g=0.719
>9, 96/234, d=0.686, g=0.720
>9, 97/234, d=0.687, g=0.715
>9, 98/234, d=0.693, g=0.694
>9, 99/234, d=0.689, g=0.701
>9, 100/234, d=0.688, g=0.692
>9, 101/234, d=0.689, g=0.685
>9, 102/234, d=0.690, g=0.671
>9, 103/234, d=0.688, g=0.677
>9, 104/234, d=0.696, g=0.668
>9, 105/234, d=0.680, g=0.676
>9, 106/234, d=0.696, g=0.671
>9, 107/234, d=0.692, g=0.683
>9, 108/234, d=0.685, g=0.699
>9, 1

In [ ]:
# example of loading the generator model and generating images
from keras.models import load_model
from numpy.random import randn
from matplotlib import pyplot
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input
 
# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, n):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	pyplot.show()
 
# load model
model = load_model('generator_model_100.h5')
# generate images
latent_points = generate_latent_points(100, 25)
# generate images
X = model.predict(latent_points)
# plot the result
save_plot(X, 5)

In [ ]:
# load model
model = load_model('generator_model_100.h5')
# all 0s
vector = asarray([[0.0 for _ in range(100)]])
# generate image
X = model.predict(vector)
# plot the result
pyplot.imshow(X[0, :, :, 0], cmap='gray_r')
pyplot.show()